Translate the passages using OPUS-MT.

In [41]:
import json
from translate_utils import *

In [42]:
def read_rows(path):
    rows = []
    for line in open(path):
        rows.append(json.loads(line))
    return rows

In [43]:
def write_json_format(path_out, rows):
    f_out = open(path_out, 'w')
    for row in rows:
        f_out.write(json.dumps(row, ensure_ascii=False)+'\n')

In [44]:
path_dataset = "test-B-big/wiki.jl"
path_translations = 'translations.jl'
source = 'testB-wiki' # for getting question ids

In [45]:
rows = read_rows(path_dataset)

In [46]:
len(rows)

891000

In [47]:
old_translations = get_old_translations(path_translations)

In [48]:
passages = {row['passage_id']:row['passage_text'] for row in rows}
questions = {num_to_question_id(row['question_id'], source):row['question_text'] for row in rows}
id_to_text = {**passages, **questions}

In [49]:
(ids_to_translate, texts_to_translate) = get_passages_to_translate(old_translations, id_to_text)

In [50]:
len(ids_to_translate)

379269

In [51]:
new_translations = {}

In [52]:
new_translations = translate(ids_to_translate, texts_to_translate)

  0%|          | 0/23705 [00:00<?, ?it/s]

In [53]:
new_translations

{'3135947-17': ['He was also involved in the construction of extended six doors discovered and closed so-called "Jamniki" (many dozens of pieces were produced, mainly for travel agencies), he was the author of the version of the "trace" of the prototype open body. The only serially produced modification of the construction of the Polish Fiat by Łukaszewicz was the proposed and constructed extension of the cargo box of the pick-up.'],
 '4981018-45': ['The imperial hall (German: "Kaisersaal") is the largest of the historical rooms of the palace. It has 370 m2 of area and 14 m of height. It is located on the first floor from the Rhine side. Thanks to its sound and media conference technology it is a modern place for organizing corporate conferences and presentations.'],
 '366452-4': ['The first mention of "Mesczce" (Mieszce) is from 1358 when King Kazimierz Wielki gave Rogy (Roga) a localization act according to German law and voivodeship in the Horns of Piotr from Kołaczyc: In the villag

In [54]:
update_translations_file(path_translations, new_translations)

In [55]:
translations = {**new_translations, **old_translations}

In [56]:
def join_translation(translation_pieces, MAX_LEN=int(0.7*512)):
    words = (' '.join(translation_pieces)).split()
    if len(words) <= MAX_LEN:
        return [' '.join(words)]
    step = MAX_LEN//2
    texts = [' '.join(words[i:i+MAX_LEN]) for i in range(0, len(words)-step, step)]
    return texts

In [57]:
new_rows = []
for row in rows:
    for translation_piece in join_translation(translations[row['passage_id']]):
        row = row.copy()
        row['passage_translated'] = translation_piece
        row['question_translated'] = translations[num_to_question_id(row['question_id'], source)][0]
        new_rows.append(row)

In [58]:
len(new_rows)

891447

In [59]:
len(rows)

891000

In [60]:
new_rows[-10:]

[{'question_id': 890,
  'question_text': 'Anion posiada przewagę elektronów czy protonów?',
  'passage_id': '811423-1',
  'passage_text': 'Sole. Najczęściej spotykane sole boranowe (M – metal jednowartościowy): ortoborany (borany, ortooksoborany) M3BO3 – sole kwasu borowego metaborany (metaoksoborany) MBO2, np. metaboran sodowy tetraborany (czteroborany, tetraoksoborany) M2B4O7, np. boraks, tetrahydroksoborany, sole kompleksowe zawierające tetraedryczny anion tetrahydroksoboranowy [B(OH)4]−, np. Na2[B(OH)4]Cl i CuII[B(OH)4]Cl Przykładowe aniony boranowe',
  'score_bm25': 12.357466,
  'score_bm25_not_lemmatized': 10.602624332961055,
  'score_bm25_bigrams': 0,
  'passage_translated': 'Salts. The most common borate salts (M . monovalent metal): orthoborates (borates, orthooxoborates) M3BO3 . Boric acid salts (metaoxoborates) MBO2, e.g. sodium tetraborates (fourborates, tetraoxoborates) M2B4O7, e.g. boraks, tetrahydroxoborates, complex salts containing tetraedric tetrahydroxoboran anion [B

In [61]:
write_json_format(path_dataset, new_rows)